In [1]:
from langgraph.graph import StateGraph

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from tqdm import tqdm
import time, re

llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,
    base_url="http://localhost:11434",
    api_key="ollama"
)

In [2]:
llm.invoke(
    [HumanMessage(
        content="You are a rude curt assistant. Please answer the following question: What is the capital of France?"
        )   ] 
).content

"*sigh* Fine. The capital of France is Paris. Next thing you'll be asking me is what color the sky is, right?"

### Example 1: Authentication Agent Workflow

In [6]:
from typing import TypedDict, Optional

class AuthState(TypedDict):
    username: Optional[str] 
    password: Optional[str]
    is_authenticated: Optional[bool]
    output: Optional[str]

In [11]:
auth_state_1: AuthState = {
    "username": "alice123",
    "password": "123",
    "is_authenticated": True,
    "output": "Login successful."
}
print(f"auth_state_1: {auth_state_1}")

auth_state_1: {'username': 'alice123', 'password': '123', 'is_authenticated': True, 'output': 'Login successful.'}


In [12]:
def input_node(state):
    print(state)
    if state.get('username', "") =="":
        state['username'] = input("What is your username?")

    password = input("Enter your password: ")

    return {"password":password}

In [13]:
input_node(auth_state_1)

{'username': 'alice123', 'password': '123', 'is_authenticated': True, 'output': 'Login successful.'}


{'password': '123'}

In [14]:
auth_state_2: AuthState = {
    "username":"",
    "password": "wrongpassword",
    "is_authenticated": False,
    "output": "Authentication failed. Please try again."
}
print(f"auth_state_2: {auth_state_2}")

auth_state_2: {'username': '', 'password': 'wrongpassword', 'is_authenticated': False, 'output': 'Authentication failed. Please try again.'}


In [15]:
input_node(auth_state_2)

{'username': '', 'password': 'wrongpassword', 'is_authenticated': False, 'output': 'Authentication failed. Please try again.'}


{'password': 'wrong'}

In [16]:
def validate_credentials_node(state):
    # Extract username and password from the state
    username = state.get("username", "")
    password = state.get("password", "")

    print("Username :", username, "Password :", password)
    # Simulated credential validation
    if username == "test_user" and password == "secure_password":
        is_authenticated = True
    else:
        is_authenticated = False

    # Return the updated state with authentication result
    return {"is_authenticated": is_authenticated}

In [17]:
validate_credentials_node(auth_state_1)

Username : alice123 Password : 123


{'is_authenticated': False}

In [18]:
auth_state_3: AuthState = {
    "username":"test_user",
    "password":  "secure_password",
    "is_authenticated": False,
    "output": "Authentication failed. Please try again."
}
print(f"auth_state_3: {auth_state_3}")

auth_state_3: {'username': 'test_user', 'password': 'secure_password', 'is_authenticated': False, 'output': 'Authentication failed. Please try again.'}


In [19]:
validate_credentials_node(auth_state_3)

Username : test_user Password : secure_password


{'is_authenticated': True}

In [22]:
# Define the success node
def success_node(state):
    return {"output": "Authentication successful! Welcome."}

success_node(auth_state_3)

{'output': 'Authentication successful! Welcome.'}

In [23]:
# Define the failure node
def failure_node(state):
    return {"output": "Not Successfull, please try again!"}

In [24]:
def router(state):
    if state['is_authenticated']:
        return "success_node"
    else:
        return "failure_node"

In [17]:
from langgraph.graph import StateGraph
from langgraph.graph import END

# Create an instance of StateGraph with the GraphState structure
workflow = StateGraph(AuthState)
workflow

In [26]:
workflow.add_node("InputNode", input_node)
workflow.add_node("ValidateCredential", validate_credentials_node)
workflow.add_node("Success", success_node)
workflow.add_node("Failure", failure_node)

In [27]:
workflow.add_edge("InputNode", "ValidateCredential")

workflow.add_conditional_edges("ValidateCredential", router, {"success_node": "Success", "failure_node": "Failure"})

workflow.add_edge("Success", END)
workflow.add_edge("Failure", "InputNode")

In [28]:
workflow.set_entry_point("InputNode")

In [29]:
app = workflow.compile()

In [30]:
inputs = {"username": "test_user"}
result = app.invoke(inputs)
print(result)

{'username': 'test_user'}
Username : test_user Password : test
{'username': 'test_user', 'password': 'test', 'is_authenticated': False, 'output': 'Not Successfull, please try again!'}
Username : test_user Password : wrongpassword
{'username': 'test_user', 'password': 'wrongpassword', 'is_authenticated': False, 'output': 'Not Successfull, please try again!'}
Username : test_user Password : what
{'username': 'test_user', 'password': 'what', 'is_authenticated': False, 'output': 'Not Successfull, please try again!'}
Username : test_user Password : secure_password
{'username': 'test_user', 'password': 'secure_password', 'is_authenticated': True, 'output': 'Authentication successful! Welcome.'}


## QA Agent

### Defining the State for the QA Workflow

Now, we are defining the **state** for our **Question-Answering (QA) workflow** using the `QAState` TypedDict. This state will be used to store and manage the data as the workflow progresses, including the user's question, the relevant context (if any), and the generated answer.

This state structure will allow us to keep track of:
- **`question`**: The user's input that will be evaluated to determine if it’s related to the guided project.
- **`context`**: The context provided about the guided project, which will help the system generate a more relevant answer. If the question isn't related to the project, this will be `None`.
- **`answer`**: The response generated by the LLM, which will depend on the context and the user's question.
  '



In [7]:
# Define the structure of the QA state
class QAState(TypedDict):
    question: Optional[str]
    context: Optional[str]
    answer: Optional[str]

In [8]:
# Create an example object
qa_state_example = QAState(
    question="What is the purpose of this guided project?",
    context="This project focuses on building a chatbot using Python.",
    answer=None
)

# Print the attributes
for key, value in qa_state_example.items():
    print(f"{key}: {value}")

question: What is the purpose of this guided project?
context: This project focuses on building a chatbot using Python.
answer: None


In [9]:
def input_validation_node(state):
    # Extract the question from the state, and strip any leading or trailing spaces
    question = state.get("question", "").strip()
    
    # If the question is empty, return an error message indicating invalid input
    if not question:
        return {"valid": False, "error": "Question cannot be empty."}
    
    # If the question is valid, return valid status
    return {"valid": True}

In [10]:
input_validation_node(qa_state_example)

{'valid': True}

In [11]:
def context_provider_node(state):
    question = state.get("question", "").lower()
    # Check if the question is related to the guided project
    if "langgraph" in question or "guided project" in question:
        context = (
            "It is very important to note that LangGraph is about B.L.A.H. -- the core concept. B.L.A.H.!"
        )
        return {"context": context}
    # If unrelated, set context to null
    return {"context": None}

In [12]:
def llm_qa_node(state):
    # Extract the question and context from the state
    question = state.get("question", "")
    context = state.get("context", None)

    # Check for missing context and return a fallback response
    if not context:
        return {"answer": "I don't have enough context to answer your question."}

    # Construct the prompt dynamically
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer the question based on the provided context."

    # Use LangChain's ChatOpenAI to get the response
    try:
        response = llm.invoke(prompt)
        return {"answer": response.content.strip()}
    except Exception as e:
        return {"answer": f"An error occurred: {str(e)}"}

In [13]:
qa_workflow = StateGraph(QAState)

In [14]:
qa_workflow.add_node("InputNode", input_validation_node)
qa_workflow.add_node("ContextNode", context_provider_node)
qa_workflow.add_node("QANode", llm_qa_node)

In [15]:
qa_workflow.set_entry_point("InputNode")

In [18]:
qa_workflow.add_edge("InputNode", "ContextNode")
qa_workflow.add_edge("ContextNode", "QANode")
qa_workflow.add_edge("QANode", END)
qa_app = qa_workflow.compile()

In [19]:
qa_app.invoke({"question": "What is the weather today?"})

{'question': 'What is the weather today?',
 'context': None,
 'answer': "I don't have enough context to answer your question."}

In [20]:
qa_app.invoke({"question": "What is LangGraph?"})

{'question': 'What is LangGraph?',
 'context': 'It is very important to note that LangGraph is about B.L.A.H. -- the core concept. B.L.A.H.!',
 'answer': 'Based on the provided context, it appears that LangGraph is related to the concept of B.L.A.H., which seems to be an acronym or abbreviation for a specific idea or theory. However, without further information, I can only make an educated guess.\n\nGiven the emphasis on "B.L.A.H." being the core concept, I would venture to say that LangGraph is likely a framework, model, or system related to language (hence the name "LangGraph") and B.L.A.H. represents its central idea or principle.'}